# Курс Python. Занятие 8. Практика по нейронным сетям

[Ссылка на блокнот в Google Drive для открытия в Google Colab](https://drive.google.com/open?id=1kmVD6AlSY3gniUqREss-ocl-Iimc47Us)

## Часть 0. Нейронные сети и фреймворк PyTorch

### Алгоритм построения нейронных сетей (моделей)
1. Подготовить наборы данных (**Datasets**) для обучения (**Train**), валидации (**Validation**) и проверки (**Test**) модели.
2. Обеспечить механизм загрузки данных из набора (**Dataloaders**).
3. Задать архитектуру нейронной сети = модель машинного обучения (**Model**).
4. Определить прямой проход (**Forward**) по нейронной сети.
5. Задать функцию ошибки (**Loss Function**) ([ссылка на перечень основных](https://medium.com/udacity-pytorch-challengers/a-brief-overview-of-loss-functions-in-pytorch-c0ddb78068f7)).
6. Задать оптимизационный алгоритм (**Optimizer**) для параметров модели ([ссылка на документацию](https://pytorch.org/docs/master/optim.html)).

### Алгоритм обучения нейронных сетей (моделей)
1. Цикл по количеству эпох (**Epoch**) - полный проход по всему тренировочному набору данных.
2. Цикл по частям набора данных (**Batches**) - входам и выходам (**Inputs** и **Outputs** или **Labels**).
3. Посчитать результат прямого прохода - выход (**Model_Output**) или предсказание модели (**Predict**)
4. Посчитать значение функции ошибки.
5. Обнулить градиенты.
6. Посчитать градиенты от ошибки (**Loss Backward**).
7. Обновить параметры модели через шаг оптимизационного алгоритма (**Optimizer Step**).
8. После обучения проверить модель на тестовом наборе данных.

### PyTorch: Тензоры (torch.Tensor)
Тензоры в PyTorch - многомерные массивы, похожие на массивы Numpy, которые реализуют возможность вычисления градиентов и расчётов на видеокартах с поддержкой CUDA.

### PyTorch: Модели (torch.nn.Module)
Модели в PyTorch - специальные классы для задания нейронных сетей или их частей, позволяющие автоматически создавать необходимое количество параметров и отслеживать их градиенты.<br>
Модели строятся из объектов-слоёв (**torch.nn.Layer**) и определения прямого прохода по сети (forward), к выходам слоёв могут применяться функции (**torch.nn.functional**).

### PyTorch: Наборы данных и загрузчики данных (torch.utils.data.Dataset и torch.utils.data.DataLoader)
Для работы с входными данными для моделей предусмотрены два основных класса: Dataset и DataLoader. <br>
Набор данных определяется классом Dataset, позволяющий поместить в себя всю логику по подготовке входных данных и реализующий методы получения элемента данных и размера набора данных (`__getitem__ и __len__`).<br>
Работа с большими объёмами данных и получения их порциями (batch) определяется классом DataLoader, который определяет размер батча, стратегию получения батчей и т.д.

### Подготовка для работы: импорты, задание наборов данных и т.д.

In [ ]:
%matplotlib inline

import torch             # основной пакет PyTorch
import torchvision       # пакет с утилитами для компьютерного зрения (наборы данных, обученные модели и т.д.)

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Зададим устройство, на котором проводить расчёты
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Зададим размер порции данных
BATCH_SIZE = 256

# Загрузим встроенный в PyTorch MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='data', train=False, transform=torchvision.transforms.ToTensor())

# Создадим специальный класс DataLoader, который будет подавать изображения порциями по batch_size
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Отобразим несколько изображений

size = 28
nrows, ncols = 8, 12
images = np.zeros((size * nrows, size * ncols))
for i in range(nrows):
    for j in range(ncols):
        index = i * ncols + j
        image = train_dataset[index][0]
        images[i * size : i * size + size, j * size : j * size + size] = image
        
fig, ax = plt.subplots()
ax.imshow(images, cmap='gray')
ax.set_axis_off()

## Часть 1. Полносвязные нейронные сети (Fully-Connected Neural Networks, FCNN)
Полносвязные нейронные сети - сети прямого распространения (Feed Forward Neural Network), в которых все нейроны каждого слоя связаны с нейронами предыдущего слоя. Соответственно, каждый такой слой называется полносвязным или линейным (**torch.nn.Linear**). ![](https://cdn-images-1.medium.com/max/1600/0*BinB4K8AxFwMKDbp)

Такие сети отличаются большим количеством параметров.

In [ ]:
# Зададим модель полносвязной сети

class FullyConnectedNetwork(torch.nn.Module):
    
    # Инициализация сети - задание параметров и объявление струтуры слоёв
    def __init__(self, input_size, hidden_size, num_classes):
        
        # Обязательно нужно вызвать инициализацию родителя
        super().__init__()
        
        # Создаём экземпляр класса полносвязного слоя 1
        self.fc1 = torch.nn.Linear(input_size, hidden_size) 
        
        # Создаём экземпляр класса функции активации ReLU
        self.relu = torch.nn.ReLU()
        
        # Создаём экземпляр класса полносвязного слоя 2
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)  
    
    # Прямой проход по сети
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# Инициализируем модель и переместим на устройство для расчётов
model = FullyConnectedNetwork(input_size=784, hidden_size=100, num_classes=10)
model = model.to(device)

# Зададим функцию ошибки (в нашем случае задача классификации, поэтому Кросс-Энтропия)
loss_function = torch.nn.CrossEntropyLoss()

# Зададим алгоритм оптимизации (например Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Напечатаем модель
print(model)

In [ ]:
# Обучим модель

# Зададим количество эпох - полных проходов по всему датасету
NUM_EPOCHS = 10

# Запустим цикл по эпохам
for epoch in range(1, NUM_EPOCHS + 1): 
    
    # Список для хранения ошибок на батче (batch)
    train_loss = []
    
    # Цикл по порциям (батчам) обучающих данных
    for images, labels in train_loader:
        
        # Переведём двумерные картинки в одномерные векторы с сохранением измерения batch
        # Переместим изображения и метки к ним на устройство для расчётов
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
                
        # 1. Выполним прямой проход по сети (модели)
        output = model(images)
        
        # 2. Вычислим ошибку
        loss = loss_function(output, labels)

        # 3. Обнулим градиенты у параметров
        optimizer.zero_grad()
        
        # 4. Выполним обратный проход (расчитаем градиенты)
        loss.backward()
        
        # 5. Выполним шаг по алгоритму оптимизации
        optimizer.step()
        
        # Добавим значение ошибки в список
        train_loss.append(loss.item())
    
    # Напечатаем среднюю ошибку на эпохе обучения
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss))

In [ ]:
# Тестируем модель, для этого говорим PyTorch не считать градиенты
with torch.no_grad():
    
    # Заведём переменные для посчёта правильных и всего ответов
    correct = 0
    total = 0
    
    # Цикл по всем батчам данных из тестового набора
    for images, labels in test_loader:
        
        # Приведём изображения в форму вектора и перенесём изображения и метки к ним на устройство
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Посчитаем предсказания модели по изображениям
        outputs = model(images)
                
        # Выведем индексы выходных нейронов с маскимальным откликом
        _, predicted = torch.max(outputs.data, dim=1)
        
        # Посчитаем количество правильных и всего ответов
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    # Напечатаем процент правильных ответов из всех
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

## Часть 2. Свёрточные нейронные сети (Convolutional Neural Networks, CNN)
Изображения - обычно многокональные, можно представить в следующем виде: ![](https://cdn-images-1.medium.com/max/1000/0*NI447zpXAZE5KT1X.png)
Свёртка (**Convolution**) - специальная операция над изображением, во время которой происходит следующее: ![](https://cdn-images-1.medium.com/max/1000/1*5SpP-dbwdZvw5knRDlkIeg.gif)
Подвыборка (**Pooling, Subsampling**) - специальная операция над изображением, во время которой происходит следующее: ![](https://cdn-images-1.medium.com/max/1000/0*-q55Kruj2uQcA9lu.gif)
Свёрточные нейронные сети состоят из слоёв свёртки, слоёв подвыборки и в конце полносвязный слой, который отвечает за классификацию: ![](https://neurohive.io/wp-content/uploads/2018/10/Typical_cnn-768x236-570x175.png)

In [ ]:
# Зададим модель свёрточной нейросети
class ConvolutionalNetwork(torch.nn.Module):

    def __init__(self):
        super().__init__()
        
        # Первая свёртка + активация + подвыборка (используем класс Sequential для последовательности слоёв)
        self.conv1 = torch.nn.Sequential(         # Входное изображение имеет размерность (1, 28, 28)
            torch.nn.Conv2d(
                in_channels=1,              # Количество входных каналов
                out_channels=32,            # Количество фильтров (ядер свёртки)
                kernel_size=5,              # Размер ядра
                stride=1,                   # Шаг свёртки
                padding=2,                  # Заходим за край изображения на 2 пикселя для сохранения размера
            ),                              # В итоге выходная размерность после свёртки (16, 28, 28)
            torch.nn.ReLU(),                      # Функция активации
            torch.nn.MaxPool2d(kernel_size=2),    # Подвыборка из окон 2х2 - размерность на выходе (16, 14, 14)
        )
        # Вторая свёртка + активация + подвыборка
        self.conv2 = torch.nn.Sequential(         # Входная размерность (16, 14, 14)
            torch.nn.Conv2d(32, 64, 5, 1, 2),     # Выходная размерность (32, 14, 14)
            torch.nn.ReLU(),                      # Функция активации
            torch.nn.MaxPool2d(2),                # Выходная размерность (32, 7, 7)
        )
        self.out = torch.nn.Linear(64 * 7 * 7, 10)   # Полносвязный слой с выходом на 10 классов

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # Перевести выход вектор (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

In [ ]:
# Инициализируем модель и переместим на устройство для расчётов
model = ConvolutionalNetwork().to(device)

# Зададим функцию ошибки (в нашем случае задача классификации, поэтому Кросс-Энтропия)
loss_function = torch.nn.CrossEntropyLoss()                     

# Зададим алгоритм оптимизации (например Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Напечатаем модель
print(model)

In [ ]:
# Обучим модель

# Зададим количество эпох - полных проходов по всему датасету
NUM_EPOCHS = 10

# Запустим цикл по эпохам
for epoch in range(1, NUM_EPOCHS + 1): 
    
    # Список для хранения ошибок на батче (batch)
    train_loss = []
    
    # Цикл по порциям (батчам) обучающих данных
    for images, labels in train_loader:
        
        # Переместим изображения и метки к ним на устройство для расчётов
        images = images.to(device)
        labels = labels.to(device)
                
        # 1. Выполним прямой проход по сети (модели)
        output = model(images)
        
        # 2. Вычислим ошибку
        loss = loss_function(output, labels)

        # 3. Обнулим градиенты у параметров
        optimizer.zero_grad()
        
        # 4. Выполним обратный проход (расчитаем градиенты)
        loss.backward()
        
        # 5. Выполним шаг по алгоритму оптимизации
        optimizer.step()
        
        # Добавим значение ошибки в список
        train_loss.append(loss.item())
    
    # Напечатаем среднюю ошибку на эпохе обучения
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss))

In [ ]:
# Тестируем модель, для этого говорим PyTorch не считать градиенты
with torch.no_grad():
    
    # Заведём переменные для посчёта правильных и всего ответов
    correct = 0
    total = 0
    
    # Цикл по всем батчам данных из тестового набора
    for images, labels in test_loader:
        
        # Перенесём изображения и метки к ним на устройство
        images = images.to(device)
        labels = labels.to(device)
        
        # Посчитаем предсказания модели по изображениям
        outputs = model(images)
                
        # Выведем индексы выходных нейронов с маскимальным откликом
        _, predicted = torch.max(outputs.data, dim=1)
        
        # Посчитаем количество правильных и всего ответов
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    # Напечатаем процент правильных ответов из всех
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

### Упражнение
0. Вынести в параметры модели количество ядер свёртки, размер свёртки с автоматическим расчётом padding и т.д.
1. Написать хелперы для обучения и проверки моделей.
2. Разбить датасет на обучающую и валидационную выборки и провести обучение с отслеживанием ошибок на обеих выборках.
3. Поэксперементировать с batch_size, слоями, размерами ядер свёртки, количеством ядер и т.д.

## Часть 3. Рекуррентные нейронные сети (Recurrent Neural Networks, RNN)
[Ссылка на статью](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)
![](https://cdn-images-1.medium.com/max/1000/1*o65pRKyHxhw7m8LgMbVERg.png)
![](https://cdn-images-1.medium.com/max/1000/1*wFYZpxTTiXVqncOLQd_CIQ.jpeg)
![](https://cdn-images-1.medium.com/max/1000/1*vhAfRLlaeOXZ-bruv7Ostg.png)

In [ ]:
# Гиперпараметры
sequence_length = 28    # длина последовательности = количество строк в изображении
input_size = 28         # размер элемента последовательности = количество столбцов в изображении
hidden_size = 128       # размер внутреннего слоя (количество нейронов)
num_layers = 2          # количество внутренних слоёв 
num_classes = 10        # размер выходного слоя = количество классов цифр
batch_size = train_loader.batch_size
learning_rate = 0.01    # скорость обучения

In [ ]:
# Рекуррентная нейронная сеть (много в один, many to one)
class RecurrentNetwork(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Создадим экзэмпляр класса LSTM ячейки
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Создадим экзэмпляр класса полносвязного слоя
        self.fc = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        
        # Зададим начальное состояние внутренних слоёв и ячеек памяти (положено для класса LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Прямой проход через LSTM ячейки 
        out, _ = self.lstm(x, (h0, c0))  # out: тензор размерности (batch_size, seq_length, hidden_size)
        
        # Подаём на полносвязный слой только элемент последовательности на последнем шаге - последнюю строку изображения
        out = self.fc(out[:, -1, :])
        
        return out

In [ ]:
# Инициализируем модель и переместим на устройство для расчётов
model = RecurrentNetwork(input_size, hidden_size, num_layers, num_classes).to(device)

# Зададим функцию ошибки (в нашем случае задача классификации, поэтому Кросс-Энтропия)
loss_function = torch.nn.CrossEntropyLoss()

# Зададим алгоритм оптимизации (например Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Напечатаем модель
print(model)


In [ ]:
# Обучим модель

# Зададим количество эпох - полных проходов по всему датасету
NUM_EPOCHS = 2

# Запустим цикл по эпохам
for epoch in range(1, NUM_EPOCHS + 1): 
    
    # Список для хранения ошибок на батче (batch)
    train_loss = []
    
    # Цикл по порциям (батчам) обучающих данных
    for images, labels in train_loader:
        
        # Переместим изображения и метки к ним на устройство для расчётов
        # Поскольку изображения имеют размерность (batch_size, channels=1, sequence_length, input_size),
        # а в модели требуется размерность (batch_size, sequence_length, input_size), приведём размерность в соответствие
        
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # 1. Выполним прямой проход по сети (модели)
        output = model(images)
        
        # 2. Вычислим ошибку
        loss = loss_function(output, labels)

        # 3. Обнулим градиенты у параметров
        optimizer.zero_grad()
        
        # 4. Выполним обратный проход (расчитаем градиенты)
        loss.backward()
        
        # 5. Выполним шаг по алгоритму оптимизации
        optimizer.step()
        
        # Добавим значение ошибки в список
        train_loss.append(loss.item())
    
    # Напечатаем среднюю ошибку на эпохе обучения
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss))

In [ ]:
# Тестируем модель, для этого говорим PyTorch не считать градиенты
with torch.no_grad():
    
    # Заведём переменные для посчёта правильных и всего ответов
    correct = 0
    total = 0
    
    # Цикл по всем батчам данных из тестового набора
    for images, labels in test_loader:
        
        # Перенесём изображения и метки к ним на устройство
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Посчитаем предсказания модели по изображениям
        outputs = model(images)
                
        # Выведем индексы выходных нейронов с маскимальным откликом
        _, predicted = torch.max(outputs.data, dim=1)
        
        # Посчитаем количество правильных и всего ответов
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    # Напечатаем процент правильных ответов из всех
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

## Часть 4. Автокодировшики (Autoencoders)
![](https://camo.githubusercontent.com/96616324d5683a2e1efe9c469ca645e119b90602/687474703a2f2f75706c6f61642d696d616765732e6a69616e7368752e696f2f75706c6f61645f696d616765732f333632333732302d356534363937376437663839303566392e706e673f696d6167654d6f6772322f6175746f2d6f7269656e742f7374726970253743696d61676556696577322f322f772f31323430)
![](https://blog.keras.io/img/ae/autoencoder_schema.jpg)
![](https://cdn-images-1.medium.com/max/1600/1*QM1b0gbKdMowkmyvO95DFA.png)

In [ ]:
# Зададим модель простого полносвязного автоэнкодера
class SimpleAutoencoder(torch.nn.Module):
    
    def __init__(self):
    
        super().__init__()
        
        # Зададим архитекруту энкодера (шифровщика) - используем специальный класс Sequential
        # Последовательно соеденим несколько линейных слоёв с ReLU активацией, выходной слой без активации
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(True),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(True), 
            torch.nn.Linear(64, 12), 
            torch.nn.ReLU(True), 
            torch.nn.Linear(12, 3)
        )
        
        # Зададим архитекруту декодера (дешифровщика) - используем специальный класс Sequential
        # Последовательно соеденим несколько линейных слоёв с ReLU активацией, выходной слой с гиперболическим тангенсом
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(3, 12),
            torch.nn.ReLU(True),
            torch.nn.Linear(12, 64),
            torch.nn.ReLU(True),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(True), 
            torch.nn.Linear(128, 28 * 28), 
            torch.nn.Tanh()
        )

    # Зададим прямой проход по сети - энкодер и декодер
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# Инициализируем модель и переместим на устройство для расчётов
model = SimpleAutoencoder().to(device)

# Зададим функцию ошибки (в нашем случае задача регрессии - выходной вектор приводим к входному, поэтому MSE)
loss_function = torch.nn.MSELoss()

# Зададим алгоритм оптимизации (например Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Напечатаем модель
print(model)

In [ ]:
# Обучим модель

# Зададим количество эпох - полных проходов по всему датасету
NUM_EPOCHS = 10

# Запустим цикл по эпохам
for epoch in range(1, NUM_EPOCHS + 1): 
    
    # Список для хранения ошибок на батче (batch)
    train_loss = []
    
    # Цикл по порциям (батчам) обучающих данных (метки в этой задаче не нужны)
    for images, _ in train_loader:
        
        # Представим изображения в виде векторов и перенесём изображения на устройство
        images = images.reshape(-1, 28*28).to(device)
        
        # 1. Выполним прямой проход по сети (модели)
        output = model(images)
        
        # 2. Вычислим ошибку
        loss = loss_function(output, images)

        # 3. Обнулим градиенты у параметров
        optimizer.zero_grad()
        
        # 4. Выполним обратный проход (расчитаем градиенты)
        loss.backward()
        
        # 5. Выполним шаг по алгоритму оптимизации
        optimizer.step()
        
        # Добавим значение ошибки в список
        train_loss.append(loss.item())
    
    # Напечатаем среднюю ошибку на эпохе обучения
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss))

In [ ]:
# Тестируем модель, для этого говорим PyTorch не считать градиенты
with torch.no_grad():
    
    # В этот раз будем просто считать среднюю ошибку на тестовых данных
    test_loss = []
    
    # Цикл по всем батчам данных из тестового набора
    for images, labels in test_loader:
        
        # Представим изображения в виде векторов и перенесём изображения на устройство
        images = images.reshape(-1, 28*28).to(device)
        
        # Посчитаем предсказания модели по изображениям
        outputs = model(images)
           
        loss = loss_function(outputs, images)
        
        test_loss.append(loss.item())
        
    # Напечатаем среднюю ошибку
    print('Mean error of the network on the 10000 test images:', np.mean(test_loss))

In [ ]:
# Вместо тестирования модели 

def decode(value1=0.0, value2=0.0, value3=0.0):
    vector = [value1, value2, value3]
    output = model.decoder(torch.Tensor(vector).to(device))
    plt.figure(1, figsize=(3, 3))
    plt.imshow(output.cpu().detach().numpy().squeeze().reshape(28, 28), cmap='gray')
    plt.xticks(ticks=[], labels=[])
    plt.yticks(ticks=[], labels=[])
    plt.show()

In [ ]:
import ipywidgets as widgets

widgets.interact_manual(decode, value1=(-10.0, 10.0, 0.1), value2=(-10.0, 10.0, 0.1), value3=(-10.0, 10.0, 0.1));